In [198]:
import pandas as pd
import numpy as np
import operator
import math
import random
from IPython.display import Audio, display


Questions for Christy:
1.) EAB column
2.) Disease Def column: differences between values
3.) What makes a street diverse?

In [3]:
df = pd.read_csv("Trees_Public.csv")

In [4]:
#df.columns

In [40]:
def dic(c):
    d = {}
    for i in c:
        try:
            d[i]+=1
        except:
            d[i]=1
    import operator
    d = sorted(d.items(), key=operator.itemgetter(1),reverse = True)
    return d
def dic2(c):
    a = {}
    for i in c:
        try:
            a[i]+=1
        except:
            a[i]=1
    import operator
    a = sorted(a.items(), key=operator.itemgetter(1),reverse = True)
    t=0
    for i in a:
        t+=i[1]
    d = []
    for i in a:
        d.append([i[0],float(i[1])/t])
    return d
def dic3(c):
    d={}
    for i in c:
        try:
            d[df[df['STREET']==i[0]]['HOOD'].iloc[0]]+=i[1]
        except:
            d[df[df['STREET']==i[0]]['HOOD'].iloc[0]]=i[1]
    for i in df['HOOD']:
        try:
            d[i]
        except:
            if i == i:
                d[i]=0
    d = sorted(d.items(), key=operator.itemgetter(1),reverse = True)
    return d
def dic4(c):
    d = {}
    for i in c:
        try:
            d[i]+=1
        except:
            d[i]=1
    return d

In [18]:
ash = df[df['COMMONNAME']=='Green Ash'].append(df[df['COMMONNAME']=='White Ash']) #Subset of tree dataset that only has ash trees
not_ash = df[df['COMMONNAME']!='Green Ash'] #Subset of tree dataset that only has ash trees
not_ash = not_ash[not_ash['COMMONNAME']!='White Ash']

mt_dic = dic(ash['MT'])  #dictionary of sorted values of Maintenance columns
strs = df['STREET'].append(df['TOSTR']).append(df['FROMSTR']).append(df['ONSTR']).unique()

strs_tree_counts={}
for i in df['STREET'].unique():
    e={}
    for j in df[df['STREET']==i]['COMMONNAME']:
        #e['Ash']=0
        if j !="White Ash" and j!="Green Ash":
            try:
                e[j]+=1
            except:
                e[j]=1
        else:
            try:
                e['Ash']+=1
            except:
                e['Ash']=1
    strs_tree_counts[i]=[np.sum([values for key,values in e.items()]),e]
for key,value in strs_tree_counts.items():
    for a,b in value[1].items():
        strs_tree_counts[key][1][a] = [b,float(b)/value[0]]

bad_ash = ash[ash['MT']!='No Specific Maintenance Need']
good_ash = ash[ash['MT']=='No Specific Maintenance Need']

boulder_trees = dic(df['COMMONNAME'])

bad_ash_strs =  [i for i in bad_ash['STREET'].unique()]
bad_ash_strs_counts = sorted([[i,len(bad_ash[bad_ash['STREET'] ==i])] for i in bad_ash_strs], key = lambda num:num[1], reverse = True)
good_ash_strs = [i for i in good_ash['STREET'].unique()]
only_good_ash_strs = [i for i in good_ash['STREET'].unique() if i not in bad_ash_strs]
only_bad_ash_strs = [i for i in bad_ash['STREET'].unique() if i not in good_ash_strs]
both_ash_strs = [i for i in good_ash['STREET'].unique() if i in bad_ash['STREET'].unique()]
both_ash_strs_percent = len(both_ash_strs)/len(strs)

bad_strs_percent = len(bad_ash_strs)/len(strs)
good_strs_percent = len(good_ash_strs)/len(strs)

bad_ash_tofro_strs = bad_ash['TOSTR'].append(bad_ash['FROMSTR']).unique()


at_risk_strs = [i for i in only_good_ash_strs if i in bad_ash_tofro_strs]

at_risk_strs_counts = []
for i in at_risk_strs:
    at_risk_strs_counts.append([i,len(ash[ash['STREET']==i])])
    
at_risk_strs_sorted = sorted(at_risk_strs_counts, key=lambda num:num[1], reverse = True)

at_risk_strs_d = {}
for i in at_risk_strs_sorted:
    at_risk_strs_d[i[0]]=i[1]
    
at_risk_strs_ash_dic = {}
for i in at_risk_strs_sorted:
    at_risk_strs_ash_dic[i[0]]=strs_tree_counts[i[0]][1]['Ash']
    
at_risk_hoods=dic3(at_risk_strs_sorted)
bad_ash_hoods_counts = dic3(bad_ash_strs_counts)


strs_trees_percent_dic = {}
for i in df['COMMONNAME'].unique():
    strs_trees_percent_dic[i]=np.mean([value[1][i][1] for key,value in strs_tree_counts.items() if i in value[1]])
t=[value[1]['Ash'][1] for key,value in strs_tree_counts.items() if 'Ash'in value[1]]
strs_trees_percent_dic['Ash']=np.mean(t)
strs_trees_percent_dic_sorted = sorted(strs_trees_percent_dic.items(), key=operator.itemgetter(1),reverse = True)

strs_trees_percent_dic_2 = {}
for i in df['COMMONNAME'].unique():
    t = []
    for key,value in strs_tree_counts.items():
        if i in value[1]:
            t.append(np.mean(value[1][i][1]))
        else:
            t.append(0)
    strs_trees_percent_dic_2[i]=np.mean(t)
t=[]
for key,value in strs_tree_counts.items():
    if 'Ash' in value[1]:
        t.append(value[1]['Ash'][1])
    else:
        t.append(0)
strs_trees_percent_dic_2['Ash']=np.mean(t)
strs_trees_percent_dic_sorted_2 = sorted(strs_trees_percent_dic_2.items(), key=operator.itemgetter(1),reverse = True)

high_percents=[]
for j in df['COMMONNAME'].unique():
    x = df[df['COMMONNAME']==j]
    for i in strs:
        try:
            if(strs_tree_counts[i][1][j][0]>50 and strs_tree_counts[i][1][j][1]>.2):
                high_percents.append([i,j,strs_tree_counts[i][1][j][0],strs_tree_counts[i][1][j][1]])
        except:
            #print('yo')
            pass
high_percents=sorted(high_percents,key=lambda x:x[3],reverse = True)
percent_ash = []
for i in strs:
    try:
        if(strs_tree_counts[i][1]['Ash'][0]>30):
            percent_ash.append([i,strs_tree_counts[i][1]['Ash'][0],strs_tree_counts[i][1]['Ash'][1]])
    except:
        #print('yo')
        pass
strs_trees_percent_dic_3 = {}
for i in dic(not_ash['COMMONNAME']):
    strs_trees_percent_dic_3[i[0]]=i[1]/len(not_ash)
strs_trees_percent_dic_sorted_3 = sorted(strs_trees_percent_dic_3.items(), key=operator.itemgetter(1),reverse = True)

replacements = {}
num_ash = len(ash)
for key,value in strs_trees_percent_dic_3.items():
    replacements[key]=num_ash*value

percent_ash_sorted = sorted(percent_ash,key=lambda x:x[2],reverse = True)

dbh_m = np.mean(ash['DBH'])
dbh_s = np.std(ash['DBH'])
d_95 = dic(ash[ash['DBH']>(dbh_m+2*dbh_s)]['STREET'])[:15]

ash_weighted_streets = {}
for i in strs:
    try:
        ash_weighted_streets[i]=strs_tree_counts[i][0]*strs_tree_counts[i][1]['Ash'][1]
    except:
        pass
ash_weighted_streets_sorted = sorted(ash_weighted_streets.items(),key=operator.itemgetter(1),reverse=True)

'''ash_weighted_streets_trees = {}
for i in ash_weighted_streets:
    if strs_tree_counts[i][1]['Ash'][1]>.08: #### grab all streets that have ash percentages of over .08
        ash_weighted_streets_trees[i]=[i,sorted(strs_tree_counts[i][1].items(),key=operator.itemgetter(1),reverse=True)[:10]]'''


/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


"ash_weighted_streets_trees = {}\nfor i in ash_weighted_streets:\n    if strs_tree_counts[i][1]['Ash'][1]>.08: #### grab all streets that have ash percentages of over .08\n        ash_weighted_streets_trees[i]=[i,sorted(strs_tree_counts[i][1].items(),key=operator.itemgetter(1),reverse=True)[:10]]"

In [19]:
help_vars = {'ash': 'filtered Data frame that only includes ash trees',
'not_ash':'self explanatory dataframe',
'mt_dic': 'sorted list of maintenance values for ash trees',
'boulder_trees': 'quantities of each tree in boulder',
'strs': 'list of all streets in boulder',

'bad_ash': 'data frame of all ash trees that need maintenance',
'bad_ash_strs': 'list of streets that have ash trees that need maintenance',
'only_bad_ash_strs': 'list of streets that have ash trees and all of those trees need maintenance',
'bad_ash_strs_counts': 'sorted list of streets and the amounts of bad ash trees they have',
'bad_strs_percent': 'percent of streets in Boulder that have bad trees',
'bad_ash_tofro_strs': 'list of streets that have another street touching it that have bad ash trees on it',
'bad_ash_hoods_counts': 'sorted list of hoods and the amount of bad ash trees that it has',

'good_ash': 'data frame of all ash trees that dont need maintenance',
'good_ash_strs': 'list of all streets that have good ash trees on it',
'only_good_ash_strs': 'list of streets that have ash trees and none of those trees need maintenance',
'good_strs_percent': 'percent of streets in Boulder that have good ash trees',

'both_ash_strs': 'list of streets that have some ash trees that need maintenance and some that dont',
'both_ash_strs_percent': 'percent of streets in boulder that have both good & bad ash trees',

'at_risk_strs': 'streets that have an ajoining street that has a tree that needs maintenance',
'at_risk_strs_d': 'alphabetized dictionary of streets that have no bad trees and the amount of bad trees on ajoining streets',
'at_risk_strs_sorted': 'list of streets that have no bad trees, sorted by how many bad trees are on ajoining streets',
'at_risk_strs_ash_dic': 'dictionary of streets that have neighboring streets with bad ash trees, and the amount of ash trees and the percentage of ash tree makeup of that street',
'at_risk_hoods': 'list of hoods, sorted by the amount of at risk streets they have',

'strs_trees_percent_dic': 'alphabetized dictionary of each tree, and the average percentage that the tree makes up for streets that have that tree',
'strs_trees_percent_dic_sorted': 'list of each tree, sorted by the average percentage that the tree makes up for streets that have that tree',

'strs_trees_percent_dic_2': 'alphabetized dictionary of each tree, and the average percentage of trees that that tree makes up for every street',
'strs_trees_percent_dic_sorted_2': 'list of each tree, sorted by average percentage of trees that that tree makes up for every street',

'strs_trees_percent_dic_3': 'alphabetized dictionary of each tree without ash and the average percentage of trees that that tree makes up for every street',
'strs_trees_percent_dic_sorted_3': 'above, but sorted',

             
'strs_tree_counts': 'Dictionary of each street. values are counts of trees on that street, and another dictionary that has each type of tree on that street, and the count of that tree and the percentage makeup of that tree on that street',
'high_percents':'list of [streets,tree,count-of-trees-on-street, percent tree] for streets that have super high percentages of a certain type of tree. filtered for only streets that have a lot of trees',
'percent_ash':'see above, but only for ash trees',
'percent_ash_sorted':'above, but sorted by percent',
'dbh_m':'mean of the diameters of ash trees at breast height',
'dbh_s':'stanrard deviation of diamters of ash trees at breast height',
'd_95':'list of streets that have ash trees that have diameters above the 95th percentile in size, and the counts of how many there are on that street',
'ash_weighted_streets':'list of streets and their counts of ash trees',
'ash_weighted_streets_sorted':'above, but sorted by count',
'ash_weighted_streets_trees':'dictionary. Keys are street names. values are list, first value in list is count of ash trees. Second value in list is a list of [type of tree, [count of that tree, percentage of that tree on street]]'}







In [21]:
len(not_ash)

44960

In [23]:
len(ash)

5709

In [27]:
np.sum([i[1] for i in dic(not_ash['COMMONNAME'])])

44960

In [42]:
t = [[i[0],i[1],float(i[1])/len(not_ash)+np.sum([j[2] for j in t])] for i in dic(not_ash['COMMONNAME'])]

In [44]:
t

[['Siberian Elm', 4236, 1.0942170818505339],
 ['Common Honeylocust', 3664, 1.0814946619217083],
 ['Flowering Crabapple', 2723, 1.0605649466192171],
 ['Blue Spruce', 1916, 1.0426156583629893],
 ['Austrian Pine', 1808, 1.0402135231316727],
 ['Silver Maple', 1646, 1.0366103202846975],
 ['Native Cottonwood', 1457, 1.0324065836298932],
 ['Northern Hackberry', 1222, 1.0271797153024911],
 ['American Linden', 1093, 1.0243104982206406],
 ['Ponderosa Pine', 1085, 1.0241325622775801],
 ['Pinon Pine', 1050, 1.0233540925266904],
 ['Norway Maple', 1007, 1.0223976868327402],
 ['Swamp White Oak', 961, 1.0213745551601423],
 ['Western Catalpa', 871, 1.0193727758007118],
 ['Russian-olive', 854, 1.0189946619217083],
 ['Boxelder', 850, 1.0189056939501779],
 ['Plains Cottonwood', 824, 1.0183274021352313],
 ['Flowering Pear', 821, 1.0182606761565836],
 ['Littleleaf Linden', 718, 1.0159697508896797],
 ['Quaking Aspen', 657, 1.0146129893238434],
 ['Juniper species', 654, 1.0145462633451958],
 ['Willow species'

In [35]:
tt=[]
num=0
for i in t:
    num+= i[2]
    tt.append([i[0],i[1],i[2],num])

In [36]:
tt

[['Siberian Elm', 4236, 0.094217081850533804, 0.094217081850533804],
 ['Common Honeylocust', 3664, 0.081494661921708192, 0.175711743772242],
 ['Flowering Crabapple', 2723, 0.060564946619217082, 0.23627669039145907],
 ['Blue Spruce', 1916, 0.042615658362989327, 0.2788923487544484],
 ['Austrian Pine', 1808, 0.040213523131672597, 0.319105871886121],
 ['Silver Maple', 1646, 0.03661032028469751, 0.35571619217081851],
 ['Native Cottonwood', 1457, 0.032406583629893237, 0.38812277580071175],
 ['Northern Hackberry', 1222, 0.027179715302491102, 0.41530249110320283],
 ['American Linden', 1093, 0.024310498220640568, 0.43961298932384341],
 ['Ponderosa Pine', 1085, 0.02413256227758007, 0.4637455516014235],
 ['Pinon Pine', 1050, 0.023354092526690393, 0.48709964412811391],
 ['Norway Maple', 1007, 0.022397686832740214, 0.50949733096085414],
 ['Swamp White Oak', 961, 0.021374555160142347, 0.53087188612099645],
 ['Western Catalpa', 871, 0.019372775800711745, 0.55024466192170818],
 ['Russian-olive', 854, 

In [37]:
ttt = {}
for i in tt:
    ttt[i[0]]=[i[1],i[2],i[3]]

In [38]:
ttt

{'Siberian Elm': [4236, 0.094217081850533804, 0.094217081850533804],
 'Common Honeylocust': [3664, 0.081494661921708192, 0.175711743772242],
 'Flowering Crabapple': [2723, 0.060564946619217082, 0.23627669039145907],
 'Blue Spruce': [1916, 0.042615658362989327, 0.2788923487544484],
 'Austrian Pine': [1808, 0.040213523131672597, 0.319105871886121],
 'Silver Maple': [1646, 0.03661032028469751, 0.35571619217081851],
 'Native Cottonwood': [1457, 0.032406583629893237, 0.38812277580071175],
 'Northern Hackberry': [1222, 0.027179715302491102, 0.41530249110320283],
 'American Linden': [1093, 0.024310498220640568, 0.43961298932384341],
 'Ponderosa Pine': [1085, 0.02413256227758007, 0.4637455516014235],
 'Pinon Pine': [1050, 0.023354092526690393, 0.48709964412811391],
 'Norway Maple': [1007, 0.022397686832740214, 0.50949733096085414],
 'Swamp White Oak': [961, 0.021374555160142347, 0.53087188612099645],
 'Western Catalpa': [871, 0.019372775800711745, 0.55024466192170818],
 'Russian-olive': [854, 

In [47]:
dic(df['SUFFIX'])

[(' ', 41077), ('X', 9589), ('x', 1), ('4635', 1), ('pl', 1)]

In [45]:
df.head(1)

FID       ID           UNIQUEID  ADDRESS SUFFIX   STREET    ONSTR  \
0    0  59084.0  JS 20150203131235      601         10TH ST  10TH ST   

       FROMSTR       TOSTR   SIDE  ...       GUARDSZ         EAB  \
0  BASELINE RD  CUL-DE-SAC  Front  ...    Unassigned  Unassigned   

              INV_DATE INV_TIME           INSPECT_DT INSPECT_TM NOTES ACTIVE  \
0  2015-02-03 00:00:00           2015-07-10 00:00:00   11:40:29          1.0   

    COMMONNAME  GENUS  
0  Blue Spruce  Picea  

[1 rows x 39 columns]

In [46]:
df.columns

Index(['FID', 'ID', 'UNIQUEID', 'ADDRESS', 'SUFFIX', 'STREET', 'ONSTR',
       'FROMSTR', 'TOSTR', 'SIDE', 'SITE', 'SPP', 'DBH', 'CULTIVAR',
       'CONDITION', 'INSPECT', 'MT', 'MT2', 'TTYPE', 'MEMTREE', 'DISEASEDEF',
       'STRUTDEFEC', 'LOCTYPE', 'SITECAT', 'HOOD', 'LOCATION', 'JURISDIC',
       'TSIP', 'GRATESZ', 'GUARDSZ', 'EAB', 'INV_DATE', 'INV_TIME',
       'INSPECT_DT', 'INSPECT_TM', 'NOTES', 'ACTIVE', 'COMMONNAME', 'GENUS'],
      dtype='object')

In [48]:
d_95

[('9TH ST', 35),
 ('ARAPAHOE AV', 22),
 ('MAPLETON AV', 15),
 ('LINCOLN PL', 10),
 ('BLUFF ST', 9),
 ('PINE ST', 8),
 ('MARINE ST', 7),
 ('11TH ST', 6),
 ('13TH ST', 6),
 ('4TH ST', 6),
 ('BROADWAY', 6),
 ('JUNIPER AV', 6),
 ('GROVE ST', 5),
 ('LINDEN PARK DR', 5),
 ('MANHATTAN DR', 5)]

In [50]:
a = ash[ash['DBH']>dbh_m+2*dbh_s]

In [57]:
dic(a[a['STREET']=='9TH ST']['ADDRESS'])

[(1201, 31), (2801, 4)]

In [58]:
dic(a[a['STREET']=='ARAPAHOE AV']['ADDRESS'])

[(5710, 22)]

In [59]:
dic(ash[ash['STREET']=='WHITE PL']['ADDRESS'])

[(5444, 17), (5401, 10), (5400, 4), (5490, 4)]

In [61]:
dic(ash[ash['STREET']=='LINDEN PARK DR']['ADDRESS'])

[(701, 102), (0, 41), (700, 5), (666, 1)]

In [104]:
num_left = len(ash)

In [107]:
round(10.12,0)

10.0

In [146]:
left.head(10)

p  quant       sum
type                                          
Siberian Elm         0.094217    538  0.094217
Common Honeylocust   0.081495    466  0.175712
Flowering Crabapple  0.060565    346  0.236277
Blue Spruce          0.042616    243  0.278892
Austrian Pine        0.040214    230  0.319106
Silver Maple         0.036610    209  0.355716
Native Cottonwood    0.032407    185  0.388123
Northern Hackberry   0.027180    155  0.415302
American Linden      0.024310    139  0.439613
Ponderosa Pine       0.024133    138  0.463746

In [160]:
for i in left['sum']:
    print(i)
    break

0.0942170818505


In [158]:
left.head(1)

p  quant       sum
type                                   
Siberian Elm  0.094217    538  0.094217

In [264]:
not_ash = not_ash[not_ash['COMMONNAME']==not_ash['COMMONNAME']]
#create list of streets
strs_r = dic(ash['STREET'])
strs_t = []
for i in strs_r:
    for j in range(i[1]):
        strs_t.append(i[0])
#create dataframe
trees_left= {}
total_trees = dic(not_ash['COMMONNAME'])
total = pd.DataFrame({'type':pd.Series([i[0] for i in total_trees]),
              'quant':pd.Series([i[1] for i in total_trees]),
              'p':pd.Series([i[1]/np.sum(pd.Series([i[1] for i in total_trees])) for i in total_trees])})
left = total
left['quant']=[int(round(i,0)) for i in left['p']*int(num_left)]

left = left.set_index('type')
#print(left.loc['Red Maple']['quant'])
left['quant']['Red Maple'] = left['quant']['Red Maple']+1
#print(left.loc['Red Maple']['quant'])
left['quant']['Chokecherry'] = left['quant']['Chokecherry']+1
left['quant']['Norway Maple'] = left['quant']['Norway Maple']+1
left['quant']['Common Honeylocust'] = left['quant']['Common Honeylocust']+1
left['quant']['Kentucky Coffeetree'] = left['quant']['Kentucky Coffeetree']+1
#print(str(np.sum(left['quant']))+" "+str(num_left))

t = 0
tt = []
for i in left['p']:
    tt.append(i+t)
    t+=i
left['sum']=tt 

suggestions2 = {}
for street in strs_t:
    #pick random tree
    r = random.random()
    tree = ""
    for j in left['sum']:
        if r <j:
            tree = list(left[left['sum']==j].index)[0]
            #print(tree)
            #print(np.sum(left['quant']))
            break
    try:
        suggestions2[street].append(tree)
    except:
        suggestions2[street]=[tree]
    
    left['quant'][tree] = left['quant'][tree]-1
    
    #remove tree
    s = np.sum(left['quant'])
    left['p']=[i/s for i in left['quant']]
    t = 0
    tt = []
    for i in left['p']:
        tt.append(i+t)
        t+=i
    left['sum']=tt   
s3 = {}
for key,value in suggestions2.items():
    s3[key]=[len(value),sorted(value)]
import json

with open('data.json', 'w') as fp:
    json.dump(s3, fp)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [220]:
for i in left.index:
    if i!=i:
        left.index
left[left.index!=left.index]

p  quant       sum
type                          
NaN   0.00327     19  0.894528

In [ ]:
display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))


In [290]:
t = {}
for key,value in strs_tree_counts.items():
    o = sorted(value[1].items(), key=operator.itemgetter(1),reverse = True)
    t[key]=[i[0] for i in o][:5]

In [292]:
with open('suggestions1.json', 'w') as fp:
    json.dump(t, fp)

In [276]:
strs_tree_counts

{'10TH ST': [426,
  {'American Elm': [9, 0.021126760563380281],
   'American Linden': [25, 0.058685446009389672],
   'Amur Maple': [2, 0.0046948356807511738],
   'Apple species': [1, 0.0023474178403755869],
   'Ash': [25, 0.058685446009389672],
   'Austrian Pine': [3, 0.0070422535211267607],
   'Blue Spruce': [13, 0.030516431924882629],
   'Boxelder': [9, 0.021126760563380281],
   'Bristlecone Pine': [1, 0.0023474178403755869],
   'Bur Oak': [7, 0.016431924882629109],
   'Catalpa species': [1, 0.0023474178403755869],
   'Chokecherry': [6, 0.014084507042253521],
   'Cockspur hawthorn': [1, 0.0023474178403755869],
   'Common Honeylocust': [21, 0.049295774647887321],
   'David Elm': [1, 0.0023474178403755869],
   'Douglas-fir': [4, 0.0093896713615023476],
   'Eastern Redcedar': [1, 0.0023474178403755869],
   'Engelmann Spruce': [3, 0.0070422535211267607],
   'English Oak': [1, 0.0023474178403755869],
   'European Beech': [1, 0.0023474178403755869],
   'European Mountainash': [3, 0.0070422

In [62]:
p = pd.read_csv('pittsburgh.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
